## 랜덤 포레스트 
정형 데이터 : 가지런히 정리되어 있는 '구조' 로 이루어져 있는 데이터 CSV, Database, Excel  
비정형 데이터 : 위와 반대되는 데이터 베이스나 엑셀로 표현하기 어려운 것들 대표적인 예시: 책, 글 같은 텍스트 데이터, 사진, 디지털 음악  
NoSQL 데이터베이스는 엑셀이나 CSV에 담기 어려운 텍스트나 JSON 데이터를 저장하는데 용이  
지금까지 배운 머신러닝 알고리즘은 정형데이터에 잘 맞ㅇ므  
가장 뛰어난 성과를 내는 알고리즘이 
## 앙상블 학습 ensemble learning  
이 알고리즘은 대부분 결정 트리를 기반으로 만들어져 있음  
비정형 데이터에는 어떤 알고리즘을 사용하는가? : 7장에서 학습할 신경망 알고리즘을 사용함  
비정형 데이터는 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로는 모델을 만들기 까다로움  
하지만 신경망 알고리즘의 발전 덕분에 사진을 인식하고 텍스트를 이해하는 모델을 만들 수 있음  
## 랜덤 포레스트 Random Forest.  
앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용되고 있음  
앙상블 학습을 적용할 때 가장 먼저 랜덤 포레스트를 시도해 보길 권장  

랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정트리(나무)의 숲을 만듦  
그 이후 결정 트리의 예측을 사용해 최종 예측을 만듦  
이 절은 사이킷 런에 구현된 앙상블 학습 알고리즘 기준으로 설명함 - 머신러닝 라이브러리 마다 구현 방식에 조금씩 차이가 있을 수 있음  
RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택함  
4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용  
다만, 회귀모형인 RandomForestRegressor는 전체 특성을 사용  

지도 학습 알고리즘에서  
분류 : 샘플을 몇 개의 클래스 중 하나로 분류하는 문제  
회귀 : 임의의 어떤 숫자를 예측하는 문제  


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [18]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [19]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [20]:
# RandomForestClassifier의 재미있는 기능
# 중복을 허용하여 부트스트랩 샘플을 만들어 결정트리를 훈련하는데 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있음 이를 OOB라고 함 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정트리를 평가할 수 있음(마치 검증세트 처럼)
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [21]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [22]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [23]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [24]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [25]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [27]:
#히스토그램 기반 그레이디언트 부스팅 Histogram-based Gradient Boosting은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘입니다. 특성을 256개의 구간으로 나누고 분할을 빠르게 찾을 수 있음 입력에 누락된 특성이 이를 따로 전처리할 필요가 없음
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.9/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [28]:
#특정 중요도를 계산하기 위해 permutation_importance()함수를 사용
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [30]:
#테스트 세트에서 특성 중요도를 계산해보기
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [31]:
#테스트세트에서 정확도 측정
hgb.score(test_input, test_target)

0.8723076923076923

그레이디언트 부스팅 알고리즘의 경우 XGBoost 라이브러리에서도 알고리즘이 구현되어 있다.  
XGBoost를 사용하여 와인 데이터의 교차 검증 점수를 확인하여 보기

In [33]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [34]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
